# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770845043636                   -0.53    9.0         
  2   -2.772141012097       -2.89       -1.31    1.0    156ms
  3   -2.772170197526       -4.53       -2.63    1.0    141ms
  4   -2.772170721972       -6.28       -4.21    2.0    164ms
  5   -2.772170723002       -8.99       -4.93    2.0    190ms
  6   -2.772170723015      -10.90       -6.16    1.0    145ms
  7   -2.772170723015      -12.35       -6.61    2.0    188ms
  8   -2.772170723015   +  -14.18       -6.94    2.0    173ms
  9   -2.772170723015      -14.05       -8.91    1.0    168ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770575981092                   -0.53    9.0         
  2   -2.772046976502       -2.83       -1.30    1.0    140ms
  3   -2.772082131844       -4.45       -2.67    1.0    154ms
  4   -2.7

1.773557934829794

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770756516091                   -0.53    8.0         
  2   -2.772050478140       -2.89       -1.32    1.0    172ms
  3   -2.772082525809       -4.49       -2.54    1.0    141ms
  4   -2.772083397884       -6.06       -3.47    2.0    174ms
  5   -2.772083414697       -7.77       -3.79    1.0    171ms
  6   -2.772083417517       -8.55       -4.56    1.0    146ms
  7   -2.772083417810       -9.53       -5.48    2.0    191ms
  8   -2.772083417810      -12.48       -6.20    1.0    153ms
  9   -2.772083417811      -12.91       -6.62    2.0    192ms
 10   -2.772083417811   +  -14.65       -6.76    1.0    164ms
 11   -2.772083417811      -14.10       -7.97    1.0    163ms
 12   -2.772083417811      -14.31       -8.11    2.0    189ms

Polarizability via ForwardDiff:       1.7725349556414487
Polarizability via finite difference: 1.773557934829794
